In [2]:
import csv
import numpy as np
import random
import math
import sys

In [3]:

'''
The loss functions shall return a scalar, which is the *average* loss of all the examples
'''

'''
For instance, the square loss of all the training examples is computed as below:

def squared_loss(train_y, pred_y):

    loss = np.mean(np.square(train_y - pred_y))

    return loss
'''

# return average loss --> use np.mean() ???
# log(1 + exp(-train_y.pred_y))
# train_y = -1/1 vector, pred_y = W.X

def logistic_loss(train_y, pred_y):
    productVector = np.multiply(train_y, pred_y)
    
    log_loss = np.log(1 + np.exp(-productVector))
    
    return np.mean(log_loss)

def hinge_loss(train_y, pred_y):
    productVector = np.multiply(train_y, pred_y)
    hinge_lossVector = np.maximum(0,1-productVector)
    return np.mean(hinge_lossVector)

In [4]:
'''
The regularizers shall compute the loss without considering the bias term in the weights
'''
def l1_reg(w):
    # take sum of absolute values 
    l1_loss = 0;
    for i in range(1,w.size):
        l1_loss += abs(w[i])
        
    return l1_loss

def l2_reg(w):
    return np.dot(w[1:], np.transpose(w[1:]))


In [5]:
# train_y is a (num_train,) +1/-1 label vector
# The complete training process should be implemented within the train_classifier function. 
# One may expect to perform sufficient number of iterations with gradient descent to update the weights in this function.

def train_classifier(train_x, train_y, learn_rate, loss, lambda_val=None, regularizer=None):
    
    # create pred_y --> goes into loss function
    
    # build expression for objective function
    
    # since the Objective function is determined only at run-time
    # compute gradient  partial derivative of each and aggregate them in a single gradient vector
    
    # w = w - learning_rate * deriv(loss function/w) -- for logistic loss only?
    
    weight_vector = np.random.rand(len(train_x[0]) + 1) # bias term included
    num_iters = 100
    h = 0.0001 # numerical_differentiation 
    for i in range(num_iters):
        current_weight = np.copy(weight_vector)
        delta_weight = np.zeros(len(train_x[0]) + 1) # produce delta_weight to update weight w = w - delta_weight
        
        predict_y = test_classifier(current_weight,train_x)
        if(lambda_val):
            current_loss = loss(train_y, predict_y) + lambda_val*regularizer(current_weight)
        else:
            current_loss = loss(train_y, predict_y)
            
        
        for index in range(len(delta_weight)):
            temp_current_weight = np.copy(current_weight)
            temp_current_weight[index] = temp_current_weight[index] + h;
                
            temp_predict_y = test_classifier(temp_current_weight,train_x)
            
            # produce loss
            if(lambda_val):
                temp_loss = loss(train_y, temp_predict_y) + lambda_val*regularizer(temp_current_weight)
            else:
                temp_loss = loss(train_y, temp_predict_y)
            
            # partial differentiation
            delta_weight[index] = (temp_loss - current_loss) / h

        # update weight vector
        weight_vector = current_weight - learn_rate*delta_weight    # W = W - dl/dW    
           
    return weight_vector

In [6]:
# retrun pred_y 
# not a binary (-1/+1) vector 
# but the inner products of weights and feature values
def test_classifier(w, test_x):
    pred_y = np.zeros(len(test_x))
    for i in range(len(test_x)):
        pred_y[i] = np.dot(w[1:], test_x[i]) + w[0]
    
    return pred_y    

In [17]:
def normalize(trainX,testX):
    # Standardize the dataset
    dataX_Transposed = trainX.transpose()
    column = 0
    for row in dataX_Transposed:
        mean = np.mean(row)
        std = np.std(row)
        for index in range(len(trainX.transpose()[0])):
            trainX[index][column] -= mean
            trainX[index][column] /= std
        for index in range(len(testX.transpose()[0])):
            testX[index][column] -= mean
            testX[index][column] /= std
        column += 1


def compute_accuracy(test_y, pred_y):
    compute_pred_y = np.copy(pred_y)
    for j in range(len(compute_pred_y)):
        if(compute_pred_y[j] < 6):
            compute_pred_y[j] = -1
        elif (compute_pred_y[j] > 6):
            compute_pred_y[j] = 1
            
    # TO-DO: add your code here
    comparison = (test_y == compute_pred_y)
    match_count = 0

    for i in range(len(comparison)):
        if (comparison[i] == True):
            match_count = match_count + 1 
            
    return (match_count/len(test_y))


In [19]:
def main():

    # Read the training data file
    szDatasetPath = 'winequality-white.csv'
    listClasses = []
    listAttrs = []
    bFirstRow = True
    with open(szDatasetPath) as csvFile:
        csvReader = csv.reader(csvFile, delimiter=',')
        for row in csvReader:
            if bFirstRow:
                bFirstRow = False
                continue
            if int(row[-1]) < 6:
                listClasses.append(-1)
                listAttrs.append(list(map(float, row[1:len(row) - 1])))
            elif int(row[-1]) > 6:
                listClasses.append(+1)
                listAttrs.append(list(map(float, row[1:len(row) - 1])))

    dataX = np.array(listAttrs)
    dataY = np.array(listClasses)
    
    
    
    # 5-fold cross-validation
	# Note: in this assignment, preprocessing the feature values will make
	# a big difference on the accuracy. 
    np.set_printoptions(precision=8)
    np.set_printoptions(suppress=True)
    np.set_printoptions(threshold=sys.maxsize)
    nNumTrainingSubset = int((len(dataX)/5))
    
    for i in range(5):
        if(i == 0):
            subdataX = np.split(dataX,[nNumTrainingSubset])
            subdataY = np.split(dataY,[nNumTrainingSubset])
            trainX = subdataX[1]
            trainY = subdataY[1]
            testX = subdataX[0]
            testY = subdataY[0]
        elif(i == 4):
            subdataX = np.split(dataX,[nNumTrainingSubset*i])
            subdataY = np.split(dataY,[nNumTrainingSubset*i])
            trainX = subdataX[0]
            trainY = subdataY[0]
            testX = subdataX[1]
            testY = subdataY[1]
        else:
            subdataX = np.split(dataX,[nNumTrainingSubset*i,nNumTrainingSubset*(i+1)])
            subdataY = np.split(dataY,[nNumTrainingSubset*i,nNumTrainingSubset*(i+1)])
            trainX = np.concatenate((subdataX[0],subdataX[2]),axis=0)
            trainY = np.concatenate((subdataY[0],subdataY[2]),axis=0)
            testX = subdataX[1]
            testY = subdataY[1]
        normalize(trainX,testX)
        

        weight_vector = train_classifier(trainX,trainY,0.001,logistic_loss)
        predict_y = test_classifier(weight_vector,testX)

        acc = compute_accuracy(testY,predict_y)
        print(weight_vector)
        print(predict_y)
        print(acc)
    # Perform feature normalization after spliting the data to training and validation set.
    # normalize()
   # (F - mean)/std_dev 
    
    # The statistics for normalization would be computed on the training set and applied on
	# training and validation set afterwards.
    
    
    # for logistic loss, no regularizer
    # Soft-margin SVM - trained with the hinge loss & l2 regularization term
    
    
    # print(weight_vector)
    
    return None

if __name__ == "__main__":

    main()

[0.71004142 0.92145379 0.60903368 0.43930673 0.73631139 0.84483596
 0.8024173  0.35914119 0.01692783 0.37535389 1.00095988]
[-1.50786385 -1.81163415 -2.32174894  1.40600205  5.84419269 -1.21050246
  2.96807662 -0.757961    2.96807662 -1.30476695  0.44649453  2.63383199
  1.5979114   0.37273776  2.00750973 -0.06318299  1.49453312  1.49453312
 -0.06097513  4.25028199  4.34181173  0.18660618  0.72849524 -2.35309119
 -1.42925503 -1.77659699  0.30342639  0.53858973 -0.82367748 -1.70624355
  3.04847384  2.37183997 -0.82367748 -1.70624355 -0.9812799  -0.23524628
 -1.10706137  0.24319157  1.51177431  3.86917125  3.86917125  3.59541801
  0.84681774  0.63487434  0.26229948 -2.34716634 -1.20893289  2.64302781
  0.76435732  0.84381441  3.21287808  2.64302781  0.76435732  1.56080152
  4.72993795  4.73444661  1.39332568  1.39332568 -1.91702885  4.73444661
  4.45224725 -2.09497017  0.71687477 -1.29587722 -0.09147183  1.68540768
  3.28340496 -3.60802532  1.68540768  5.17158725  1.47371455  1.00101887


[0.02834761 0.75187835 0.32326401 0.34308665 0.61455738 0.10073004
 0.68073435 0.11878073 0.02654006 0.78275834 0.27397942]
[ 2.95123451 -0.04333257 -0.04333257  0.88976534  0.99755385 -2.13639548
 -0.49388949  0.44991249  1.20198133  0.64104604 -1.29287806  0.05005944
  0.64104604  0.05005944  0.83209307 -1.29287806 -2.24841272 -1.91286267
 -2.10897802 -0.57599489  0.40833203  3.4482165   2.77900855  0.40833203
 -0.42692515 -3.60234624 -0.46987088  4.63155454 -0.46987088  4.63155454
  1.39426798  1.28100281  1.28100281 -0.55761943  0.54758586  0.95469006
  5.79913876 -0.61159762  0.95469006  1.70834863 -3.7818988  -3.64774164
 -3.64774164 -0.15012156 -0.84393392  0.67972383  1.08311619  2.95778248
  2.95778248 -0.51632002  0.80939476  0.80939476  2.09843645  1.12672857
  1.12672857  1.12672857  1.12672857  1.12672857  0.54264924 -1.82605757
  0.04699334  1.12672857 -2.00214773 -1.06259331 -1.10782636 -1.04231404
 -0.26521881 -1.04231404 -0.04323765 -0.46509273 -0.7268429  -0.46509273


[0.16731865 0.26372967 0.79907441 0.67451123 0.61331895 0.25258812
 0.24821733 0.29671273 0.42936017 0.75753709 0.29189672]
[-2.4729239   3.59196057  0.55961477 -0.80940919  1.85151096  3.59196057
  1.66717435 -1.45307732  0.96362774  1.62406989  1.97407007  1.66717435
  1.79412701 -0.40146736  0.47456184 -0.45835991 -0.27699629 -1.3119058
 -0.45835991 -1.3119058   0.47456184  1.06847544  2.58314185 -0.14863859
 -0.45835991 -0.27699629  1.12908824 -0.25458987  0.86817376 -2.68669229
  1.91306019 -1.07733898 -2.798058   -0.20620813 -1.15208409 -1.71788957
 -1.71788957  0.39729587 -1.4165268   0.39729587 -1.4165268  -0.6586108
  2.05233545  2.05233545  1.76242253 -1.5959369   1.27794436 -0.6603809
 -0.6603809  -0.73139747 -0.6603809   3.23280861  3.51997173  2.28713424
 -1.09195738  1.23113107 -1.0690028   1.85469347 -1.41475062 -0.54434176
 -0.54434176 -0.7587025  -3.78730622 -0.84331631 -0.7587025  -3.78730622
 -3.08835274  0.89015544  0.29594442  0.29594442 -2.25694431 -0.83626195
 -0